## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
combined_df = pd.merge(mouse_metadata, study_results, on="Mouse ID", how="outer")
# Display the data table for preview
combined_df.head()

Mouse ID Drug Regimen   Sex  Age_months  Weight (g)  Timepoint  \
0     k403     Ramicane  Male          21          16          0   
1     k403     Ramicane  Male          21          16          5   
2     k403     Ramicane  Male          21          16         10   
3     k403     Ramicane  Male          21          16         15   
4     k403     Ramicane  Male          21          16         20   

   Tumor Volume (mm3)  Metastatic Sites  
0           45.000000                 0  
1           38.825898                 0  
2           35.014271                 1  
3           34.223992                 1  
4           32.997729                 1

In [2]:
# Checking the number of mice.
combined_df["Mouse ID"].nunique()

249

In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicated = combined_df[combined_df["Mouse ID"].duplicated()]

print(duplicated["Mouse ID"].unique())

['k403' 's185' 'x401' 'm601' 'g791' 's508' 'f966' 'm546' 'z578' 'j913'
 'u364' 'n364' 'y793' 'r554' 'm957' 'c758' 't565' 'a644' 'i177' 'j989'
 'i738' 'a520' 'w914' 'r811' 'g288' 'i334' 'q610' 'd251' 'l897' 'c458'
 'b742' 'b128' 'j246' 'a411' 'j119' 'w150' 'v923' 'g316' 's710' 'l509'
 'r944' 'e662' 'u196' 'q597' 'a444' 'i557' 'r921' 'w678' 'y449' 'a203'
 'a251' 'a262' 'a275' 'a366' 'a401' 'a457' 'a492' 'a577' 'a685' 'a699'
 'a788' 'a818' 'a897' 'a963' 'b313' 'b487' 'b559' 'b759' 'b879' 'c139'
 'c264' 'c282' 'c302' 'c326' 'c402' 'c559' 'c580' 'c757' 'c766' 'c819'
 'c832' 'c895' 'c927' 'd164' 'd474' 'e213' 'e227' 'e291' 'e476' 'e584'
 'f129' 'f234' 'f278' 'f345' 'f394' 'f436' 'f545' 'f993' 'g107' 'g296'
 'g497' 'g558' 'g570' 'g867' 'g989' 'h246' 'h333' 'h531' 'i386' 'i477'
 'i635' 'i669' 'i901' 'j235' 'j296' 'j365' 'j755' 'j984' 'k210' 'k382'
 'k483' 'k510' 'k603' 'k754' 'k804' 'k862' 'k894' 'l215' 'l264' 'l471'
 'l490' 'l558' 'l661' 'l700' 'l725' 'l733' 'm133' 'm269' 'm331' 'm550'
 'm650

In [4]:
# Optional: Get all the data for the duplicate mouse ID.
data_df = combined_df.set_index(['Mouse ID', 'Timepoint'])
data_df.head(15)

Drug Regimen     Sex  Age_months  Weight (g)  \
Mouse ID Timepoint                                                
k403     0             Ramicane    Male          21          16   
         5             Ramicane    Male          21          16   
         10            Ramicane    Male          21          16   
         15            Ramicane    Male          21          16   
         20            Ramicane    Male          21          16   
         25            Ramicane    Male          21          16   
         30            Ramicane    Male          21          16   
         35            Ramicane    Male          21          16   
         40            Ramicane    Male          21          16   
         45            Ramicane    Male          21          16   
s185     0            Capomulin  Female           3          17   
         5            Capomulin  Female           3          17   
         10           Capomulin  Female           3          17   
         15           Capomulin  Female           3          17   
         20           Capomulin  Female           3          17   

                    Tumor Volume (mm3)  Metastatic Sites  
Mouse ID Timepoint                                        
k403     0                   45.000000                 0  
         5                   38.825898                 0  
         10                  35.014271                 1  
         15                  34.223992                 1  
         20                  32.997729                 1  
         25                  33.464577                 1  
         30                  31.099498                 1  
         35                  26.546993                 1  
         40                  24.365505                 1  
         45                  22.050126                 1  
s185     0                   45.000000                 0  
         5                   43.878496                 0  
         10                  37.614948                 0  
         15                  38.177232                 0  
         20                  36.866876                 0

In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
combined_df.sort_values("Timepoint")

clean_df_last = combined_df.drop_duplicates(subset=["Mouse ID"], keep="last")
clean_df_last.head()

Mouse ID Drug Regimen     Sex  Age_months  Weight (g)  Timepoint  \
9      k403     Ramicane    Male          21          16         45   
19     s185    Capomulin  Female           3          17         45   
29     x401    Capomulin  Female          16          15         45   
39     m601    Capomulin    Male          22          17         45   
49     g791     Ramicane    Male          11          16         45   

    Tumor Volume (mm3)  Metastatic Sites  
9            22.050126                 1  
19           23.343598                 1  
29           28.484033                 0  
39           28.430964                 1  
49           29.128472                 1

In [6]:
# Checking the number of mice in the clean DataFrame.
clean_df_last["Mouse ID"].nunique()

249

## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



# group by and function all the things
tumor_mean = combined_df.groupby("Drug Regimen").mean()
tumor_median = combined_df.groupby("Drug Regimen").median()
tumor_variance = combined_df.groupby("Drug Regimen").var()
tumor_SD = combined_df.groupby("Drug Regimen").std()
tumor_SEM = combined_df.groupby("Drug Regimen").sem()

# makes drug regs an array
drug_reg = combined_df["Drug Regimen"].unique()


tumor_sum_df = pd.DataFrame({"Tumor Means": tumor_mean["Tumor Volume (mm3)"],
                            "Tumor Medians": tumor_median["Tumor Volume (mm3)"],
                            "Tumor Variances": tumor_variance["Tumor Volume (mm3)"],
                            "Tumor Standard Definitions": tumor_SD["Tumor Volume (mm3)"],
                            "Tumor SEM": tumor_SEM["Tumor Volume (mm3)"],
                            })


tumor_sum_df


Tumor Means  Tumor Medians  Tumor Variances  \
Drug Regimen                                                
Capomulin       40.675741      41.557809        24.947764   
Ceftamin        52.591172      51.776157        39.290177   
Infubinol       52.884795      51.820584        43.128684   
Ketapril        55.235638      53.698743        68.553577   
Naftisol        54.331565      52.509285        66.173479   
Placebo         54.033581      52.288934        61.168083   
Propriva        52.322552      50.854632        42.351070   
Ramicane        40.216745      40.673236        23.486704   
Stelasyn        54.233149      52.431737        59.450562   
Zoniferol       53.236507      51.818479        48.533355   

              Tumor Standard Definitions  Tumor SEM  
Drug Regimen                                         
Capomulin                       4.994774   0.329346  
Ceftamin                        6.268188   0.469821  
Infubinol                       6.567243   0.492236  
Ketapril                        8.279709   0.603860  
Naftisol                        8.134708   0.596466  
Placebo                         7.821003   0.581331  
Propriva                        6.507770   0.512884  
Ramicane                        4.846308   0.320955  
Stelasyn                        7.710419   0.573111  
Zoniferol                       6.966589   0.516398

In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line

#agg the heck out of it yay
agg_df = combined_df.groupby(["Drug Regimen"]).agg(Tumor_Means=("Tumor Volume (mm3)",np.mean), 
                                                   Tumor_Medians=("Tumor Volume (mm3)",np.median),
                                                   Tumor_Variances=("Tumor Volume (mm3)",np.var),
                                                   Tumor_Standard_Definitions=("Tumor Volume (mm3)",np.std),
                                                   Tumor_SEMs=("Tumor Volume (mm3)",st.sem))
agg_df

Tumor_Means  Tumor_Medians  Tumor_Variances  \
Drug Regimen                                                
Capomulin       40.675741      41.557809        24.947764   
Ceftamin        52.591172      51.776157        39.290177   
Infubinol       52.884795      51.820584        43.128684   
Ketapril        55.235638      53.698743        68.553577   
Naftisol        54.331565      52.509285        66.173479   
Placebo         54.033581      52.288934        61.168083   
Propriva        52.322552      50.854632        42.351070   
Ramicane        40.216745      40.673236        23.486704   
Stelasyn        54.233149      52.431737        59.450562   
Zoniferol       53.236507      51.818479        48.533355   

              Tumor_Standard_Definitions  Tumor_SEMs  
Drug Regimen                                          
Capomulin                       4.994774    0.329346  
Ceftamin                        6.268188    0.469821  
Infubinol                       6.567243    0.492236  
Ketapril                        8.279709    0.603860  
Naftisol                        8.134708    0.596466  
Placebo                         7.821003    0.581331  
Propriva                        6.507770    0.512884  
Ramicane                        4.846308    0.320955  
Stelasyn                        7.710419    0.573111  
Zoniferol                       6.966589    0.516398

## Bar and Pie Charts

# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.

# clean_df

In [10]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

# clean_df

In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

# clean_df

## Quartiles, Outliers and Boxplots

In [12]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [13]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [14]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [15]:
# EXTRA CELL YOU PUT IN


# CREATED Capomulin DF
capomulin_df = combined_df[combined_df["Drug Regimen"].str.contains("Capomulin", case=False)]
capomulin_df

Mouse ID Drug Regimen     Sex  Age_months  Weight (g)  Timepoint  \
10       s185    Capomulin  Female           3          17          0   
11       s185    Capomulin  Female           3          17          5   
12       s185    Capomulin  Female           3          17         10   
13       s185    Capomulin  Female           3          17         15   
14       s185    Capomulin  Female           3          17         20   
...       ...          ...     ...         ...         ...        ...   
440      i557    Capomulin  Female           1          24         45   
1452     r157    Capomulin    Male          22          25          0   
1453     r157    Capomulin    Male          22          25          5   
1454     r157    Capomulin    Male          22          25         10   
1455     r157    Capomulin    Male          22          25         15   

      Tumor Volume (mm3)  Metastatic Sites  
10             45.000000                 0  
11             43.878496                 0  
12             37.614948                 0  
13             38.177232                 0  
14             36.866876                 0  
...                  ...               ...  
440            47.685963                 1  
1452           45.000000                 0  
1453           45.597064                 0  
1454           46.059608                 0  
1455           46.539206                 0  

[230 rows x 8 columns]

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
